**BUILD THE NEURAL NETWORK**

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

**Get device for training** (generally GPU)

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

using cuda device


**Define the class**: subclass the nn.Module and initilize le nn layers in init. Every nn.Module subclass implements the operations on input data in the forward method.

This is a simple Multi-Layer Perceptron (MLP) architecture with three layers: an input layer, a hidden layer, and an output layer. The input layer has 784 nodes (which comes from flattening the 28x28 input image), while the output layer has 10 nodes (for classification into 10 classes). The hidden layer consists of 512 nodes, and ReLU activation functions are used between every pair of layers except the output layer.

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # used to reshape the input data
      # core of the nn as a sequential stack of fully connected layers interleaved with non-linear activations
      # this architecture has 3 layers: 784-512-512-10 network, where the input is flattened (28*28) and the output is 10 classes
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Create an instance of NN, move it to device and print the structure

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [10]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)  # identifies the most likely class as the one with highest predicted probability.
print(f"predicted class: {y_pred}")

predicted class: tensor([5], device='cuda:0')


**MODEL LAYERS**

In [11]:
# take a sample minibatch of 3 images of size 28x28
input_img = torch.rand(3, 28, 28)
print(input_img.size())

torch.Size([3, 28, 28])


**nn.Flatten** layer to convert each 2D 28x28 image to a contiguous array of 784 pixel values

In [12]:
flatten = nn.Flatten()
flat_img = flatten(input_img)
print(flat_img.size())

torch.Size([3, 784])


**nn.Linear**: applies a linear transformation on the input using its stored weights and biases

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_img)
hidden1.size()

torch.Size([3, 20])

**nn.ReLU**

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1073,  0.0693,  0.4008, -0.1351,  0.0533, -0.2445,  0.0142,  0.2450,
          0.0470, -0.3732,  0.2360,  0.6228, -0.1413, -0.0463,  0.5008, -0.4194,
         -0.4302, -0.5236,  0.3025, -0.0974],
        [-0.1589, -0.2283,  0.3638, -0.0638,  0.0780,  0.0180, -0.0457, -0.1702,
          0.1087, -0.2787,  0.3073,  0.6033, -0.0831,  0.2223,  0.2262, -0.2378,
         -0.0220, -0.5302,  0.2020, -0.0533],
        [ 0.0216,  0.1599,  0.1937,  0.0721,  0.0253, -0.0800, -0.1953,  0.3428,
          0.2066,  0.0452,  0.2235,  0.6604, -0.0452, -0.0281,  0.4596,  0.1279,
         -0.1056, -0.2983,  0.6146,  0.2014]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0693, 0.4008, 0.0000, 0.0533, 0.0000, 0.0142, 0.2450, 0.0470,
         0.0000, 0.2360, 0.6228, 0.0000, 0.0000, 0.5008, 0.0000, 0.0000, 0.0000,
         0.3025, 0.0000],
        [0.0000, 0.0000, 0.3638, 0.0000, 0.0780, 0.0180, 0.0000, 0.0000, 0.1087,
         0.0000, 0.3073, 0.6033, 0.0000, 0.2223, 0.22

**nn.Sequential**

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

**nn.Softmax**

The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

**Model parameters**

Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0293,  0.0143,  0.0289,  ..., -0.0276, -0.0263,  0.0241],
        [-0.0117,  0.0029,  0.0268,  ...,  0.0169, -0.0114, -0.0277]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0163,  0.0083], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0162,  0.0338,  0.0027,  ..., -0.0111, -0.0118, -0.0169],
        [ 0.0004, -0.0077, -0.0248,  ...,  0.0044, -0.0435,  0.0331]],
       device='cuda:0', grad_fn=<Sl